## nb_plata_a_oro
### En este notebook se lleva a cabo la limpieza de los datos del archivo de plata y la creacion de nuevos, para posteriormente guardar el archivo en oro y poder sacar correlaciones y gráficas

## Imports

In [65]:
import os
from datetime import datetime
import pandas as pd
import numpy as np

## Parámetros

In [66]:
plata_path = os.path.join(os.curdir, "Data_Lake", "Plata")
oro_path = os.path.join(os.curdir, "Data_Lake", "Oro")

In [67]:
nombre_archivo_plata = "plata.parquet"
nombre_archivo_oro = "oro.parquet"

In [68]:
plata_file_path = os.path.join(plata_path, nombre_archivo_plata)
oro_file_path = os.path.join(oro_path, nombre_archivo_oro)

## Main

### Cargar archivo plata y ver columnas

In [69]:
plata_por_pedido_df = pd.read_parquet(plata_file_path)
plata_por_pedido_df.columns

Index(['_SilverTimestamp', 'order_key', 'Name', 'Risk Level', 'Source',
       'created_at_utc', 'paid_at_utc', 'cancelled_at_utc',
       'next_payment_due_at_utc', 'financial_status_bruto', 'is_paid',
       'currency', 'subtotal_cents', 'shipping_cents', 'taxes_cents',
       'duties_cents', 'discount_amount_cents', 'order_value_cents',
       'outstanding_balance_cents', 'refunded_amount_cents', 'payment_method',
       'shipping_method', 'discount_code', 'shipping_country',
       'accepts_marketing_bool', 'es_outlier_importe_total',
       'es_outlier_gasto_envio', 'es_outlier_impuestos',
       'es_outlier_aranceles', 'es_outlier_descuento',
       'es_outlier_saldo_pendiente', 'es_outlier_importe_reembolsado',
       'items_count', 'unique_skus_count', 'avg_item_price_cents'],
      dtype='object')

### Dejar solo los pedidos pagados

In [70]:
if "is_paid" in plata_por_pedido_df.columns:
    plata_para_oro_df = plata_por_pedido_df[plata_por_pedido_df["is_paid"] == True]
else:
    plata_para_oro_df = plata_por_pedido_df

In [71]:
plata_para_oro_df.columns

Index(['_SilverTimestamp', 'order_key', 'Name', 'Risk Level', 'Source',
       'created_at_utc', 'paid_at_utc', 'cancelled_at_utc',
       'next_payment_due_at_utc', 'financial_status_bruto', 'is_paid',
       'currency', 'subtotal_cents', 'shipping_cents', 'taxes_cents',
       'duties_cents', 'discount_amount_cents', 'order_value_cents',
       'outstanding_balance_cents', 'refunded_amount_cents', 'payment_method',
       'shipping_method', 'discount_code', 'shipping_country',
       'accepts_marketing_bool', 'es_outlier_importe_total',
       'es_outlier_gasto_envio', 'es_outlier_impuestos',
       'es_outlier_aranceles', 'es_outlier_descuento',
       'es_outlier_saldo_pendiente', 'es_outlier_importe_reembolsado',
       'items_count', 'unique_skus_count', 'avg_item_price_cents'],
      dtype='object')

### Eliminar los outliers gracias a las flags que creamos en plata

In [72]:
lista_posibles_flags_outliers = ["es_outlier_importe_total", "es_outlier_gasto_envio", "es_outlier_impuestos", "es_outlier_aranceles", "es_outlier_descuento", "es_outlier_saldo_pendiente", "es_outlier_importe_reembolsado"]

In [73]:
flags_presentes_en_dataframe = []
for nombre_flag in lista_posibles_flags_outliers:
    if nombre_flag in plata_para_oro_df.columns:
        flags_presentes_en_dataframe.append(nombre_flag)

In [74]:
if len(flags_presentes_en_dataframe) > 0:
    for nombre_flag in flags_presentes_en_dataframe:
        plata_para_oro_df = plata_para_oro_df[~(plata_para_oro_df[nombre_flag] == True)]

In [75]:
plata_para_oro_df.columns

Index(['_SilverTimestamp', 'order_key', 'Name', 'Risk Level', 'Source',
       'created_at_utc', 'paid_at_utc', 'cancelled_at_utc',
       'next_payment_due_at_utc', 'financial_status_bruto', 'is_paid',
       'currency', 'subtotal_cents', 'shipping_cents', 'taxes_cents',
       'duties_cents', 'discount_amount_cents', 'order_value_cents',
       'outstanding_balance_cents', 'refunded_amount_cents', 'payment_method',
       'shipping_method', 'discount_code', 'shipping_country',
       'accepts_marketing_bool', 'es_outlier_importe_total',
       'es_outlier_gasto_envio', 'es_outlier_impuestos',
       'es_outlier_aranceles', 'es_outlier_descuento',
       'es_outlier_saldo_pendiente', 'es_outlier_importe_reembolsado',
       'items_count', 'unique_skus_count', 'avg_item_price_cents'],
      dtype='object')

### Fechas

A partir de created_at_utc dejar solo fecha sin hora para agrupar por dia y asi poder agrupar por año, mes, dia de la semana,etc.


In [76]:
if "created_at_utc" in plata_para_oro_df.columns:
    serie_fecha_creacion_utc = plata_para_oro_df["created_at_utc"]

In [77]:
plata_para_oro_df["order_date_utc"] = serie_fecha_creacion_utc.dt.normalize()

In [78]:
plata_para_oro_df["order_year"] = serie_fecha_creacion_utc.dt.year.astype("Int64")
plata_para_oro_df["order_month"] = serie_fecha_creacion_utc.dt.month.astype("Int64")
plata_para_oro_df["order_weekday"] = serie_fecha_creacion_utc.dt.dayofweek.astype("Int64")  # 0 = lunes
plata_para_oro_df["order_hour"] = serie_fecha_creacion_utc.dt.hour.astype("Int64")

### Crear funcion para calcular ratios y aplicarlos

In [79]:
def calcular_ratio_simple(serie_numerador, serie_denominador):
    """
    Devuelve numerador/denominador como float.
    - Convierte ambos a numérico.
    - Si el denominador es 0 o NaN -> resultado NaN.
    """
    numerador   = pd.to_numeric(serie_numerador,   errors="coerce")
    denominador = pd.to_numeric(serie_denominador, errors="coerce").replace({0: np.nan})
    return numerador / denominador

### Calcular valor neto del pedido en céntimos

In [80]:
if ("order_value_cents" in plata_para_oro_df.columns) and ("refunded_amount_cents" in plata_para_oro_df.columns):
    plata_para_oro_df["order_value_net_cents"] = (plata_para_oro_df["order_value_cents"] - plata_para_oro_df["refunded_amount_cents"].fillna(0)).round().astype("Int64")

### Calcular ratios

In [81]:
# discount_rate = discount_amount_cents / subtotal_cents
# midela proporción de descuento aplicada respecto al valor de los artículos
if ("discount_amount_cents" in plata_para_oro_df.columns) and ("subtotal_cents" in plata_para_oro_df.columns):
    plata_para_oro_df["discount_rate"] = calcular_ratio_simple(plata_para_oro_df["discount_amount_cents"], plata_para_oro_df["subtotal_cents"])

In [82]:
# shipping_rate = shipping_cents / order_value_cents
# mide la carga del envío dentro del valor total del pedido (incluye artículos y, normalmente, impuestos).
if ("shipping_cents" in plata_para_oro_df.columns) and ("order_value_cents" in plata_para_oro_df.columns):
    plata_para_oro_df["shipping_rate"] = calcular_ratio_simple(plata_para_oro_df["shipping_cents"], plata_para_oro_df["order_value_cents"])

In [83]:
# taxes_rate = taxes_cents / subtotal_cents
# mide la tasa efectiva de impuestos (como el IVA) sobre la base de artículos
if ("taxes_cents" in plata_para_oro_df.columns) and ("subtotal_cents" in plata_para_oro_df.columns):
    plata_para_oro_df["taxes_rate"] = calcular_ratio_simple(plata_para_oro_df["taxes_cents"], plata_para_oro_df["subtotal_cents"])

In [84]:
# refund_rate = refunded_amount_cents / order_value_cents
# Calidad/posventa. Es clave para detectar problemas de producto, talla, logística, etc.
if ("refunded_amount_cents" in plata_para_oro_df.columns) and ("order_value_cents" in plata_para_oro_df.columns):
    plata_para_oro_df["refund_rate"] = calcular_ratio_simple(plata_para_oro_df["refunded_amount_cents"], plata_para_oro_df["order_value_cents"])

In [85]:
# basket_diversity_rate = unique_skus_count / items_count
# Señal de variedad por pedido
if ("unique_skus_count" in plata_para_oro_df.columns) and ("items_count" in plata_para_oro_df.columns):
    plata_para_oro_df["basket_diversity_rate"] = calcular_ratio_simple(plata_para_oro_df["unique_skus_count"], plata_para_oro_df["items_count"])

### Agrupar los metodos de pago en categorias comunes

In [86]:
sorted(plata_para_oro_df["payment_method"].dropna().unique())

['easypay',
 'klarna',
 'klarna + shopify payments',
 'monei mbway',
 'monei pay · bizum',
 'paypal express checkout',
 'paypal express checkout + shopify payments',
 'shopify payments',
 'shopify payments + easypay',
 'shopify payments + klarna',
 'shopify payments + monei pay · bizum',
 'shopify payments + paypal express checkout']

In [87]:
if "payment_method" in plata_para_oro_df.columns:
    serie_metodo_pago_limpia = (
        plata_para_oro_df["payment_method"]
        .astype("string") # dtype de cadena de pandas
        .fillna("") # evita NA y fuerza que str.contains devuelva bool "puro"
        .str.lower() # pasamos a minúsculas
        .str.strip() # quitamos espacios en extremos
    )

In [88]:
lista_condiciones_metodo_pago = [
        serie_metodo_pago_limpia.str.contains(r"\+", na=False),                 
        serie_metodo_pago_limpia.str.contains("paypal", na=False),
        serie_metodo_pago_limpia.str.contains("klarna",na=False),
        serie_metodo_pago_limpia.str.contains("monei|bizum|mbway", na=False),
        serie_metodo_pago_limpia.str.contains("easypay", na=False),
        serie_metodo_pago_limpia.str.contains("shopify payments", na=False),
    ]

In [89]:
lista_categorias_metodo_pago = [
        "Mixto",
        "PayPal",
        "Klarna",
        "MONEI",
        "EasyPay",
        "Shopify Payments",
    ]

In [90]:
plata_para_oro_df["categoria_metodo_pago"] = np.select(
        lista_condiciones_metodo_pago,
        lista_categorias_metodo_pago,
        default="Otro/Desconocido"
    )

In [91]:
plata_para_oro_df.columns

Index(['_SilverTimestamp', 'order_key', 'Name', 'Risk Level', 'Source',
       'created_at_utc', 'paid_at_utc', 'cancelled_at_utc',
       'next_payment_due_at_utc', 'financial_status_bruto', 'is_paid',
       'currency', 'subtotal_cents', 'shipping_cents', 'taxes_cents',
       'duties_cents', 'discount_amount_cents', 'order_value_cents',
       'outstanding_balance_cents', 'refunded_amount_cents', 'payment_method',
       'shipping_method', 'discount_code', 'shipping_country',
       'accepts_marketing_bool', 'es_outlier_importe_total',
       'es_outlier_gasto_envio', 'es_outlier_impuestos',
       'es_outlier_aranceles', 'es_outlier_descuento',
       'es_outlier_saldo_pendiente', 'es_outlier_importe_reembolsado',
       'items_count', 'unique_skus_count', 'avg_item_price_cents',
       'order_date_utc', 'order_year', 'order_month', 'order_weekday',
       'order_hour', 'order_value_net_cents', 'discount_rate', 'shipping_rate',
       'taxes_rate', 'refund_rate', 'basket_diversity_

### Seleccionar las 20 columnas objetivo para usar en oro

In [92]:
columnas_objetivo_20 = [
    "order_date_utc",
    "order_year",
    "order_month",
    "order_weekday",
    "order_hour",
    "categoria_metodo_pago",
    "order_value_cents",
    "order_value_net_cents",
    "subtotal_cents",
    "discount_amount_cents",
    "taxes_cents",
    "shipping_cents",
    "refunded_amount_cents",
    "items_count",
    "unique_skus_count",
    "discount_rate",
    "shipping_rate",
    "taxes_rate",
    "refund_rate",
    "basket_diversity_rate"
]

In [93]:
columnas_finales_oro = []
indice_columna_objetivo = 0
while indice_columna_objetivo < len(columnas_objetivo_20):
    nombre_columna_objetivo = columnas_objetivo_20[indice_columna_objetivo]
    existe_en_dataframe_actual = nombre_columna_objetivo in plata_para_oro_df.columns
    if existe_en_dataframe_actual:
        columnas_finales_oro.append(nombre_columna_objetivo)
    indice_columna_objetivo = indice_columna_objetivo + 1

In [ ]:
columnas_finales_oro 

['order_date_utc',
 'order_year',
 'order_month',
 'order_weekday',
 'order_hour',
 'categoria_metodo_pago',
 'order_value_cents',
 'order_value_net_cents',
 'subtotal_cents',
 'discount_amount_cents',
 'taxes_cents',
 'shipping_cents',
 'refunded_amount_cents',
 'items_count',
 'unique_skus_count',
 'discount_rate',
 'shipping_rate',
 'taxes_rate',
 'refund_rate',
 'basket_diversity_rate']

### Guardar archivo en oro

In [95]:
oro_df = plata_para_oro_df.loc[:, columnas_finales_oro]

In [96]:
oro_df = oro_df.convert_dtypes()

In [97]:
os.makedirs(oro_path, exist_ok=True)
oro_df.to_parquet(oro_file_path, index=False)